# 这个文件是下载并导入的文件

In [10]:
import GEOparse
import datetime

In [11]:
GEONAME = 'GSE6269'
# GSE63990  Profiling of bacterial respiratory infection, viral respiratory infection, and non-infectious illness
# GSE6269   Gene expression patterns in blood leukocytes discriminate patients with acute infections


In [12]:
gse = GEOparse.get_GEO(geo=GEONAME, destdir="./datasets", silent=False)

21-Apr-2023 15:29:58 DEBUG utils - Directory ./datasets already exists. Skipping.
21-Apr-2023 15:29:58 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE6nnn/GSE6269/soft/GSE6269_family.soft.gz to ./datasets\GSE6269_family.soft.gz
100%|██████████| 41.4M/41.4M [00:07<00:00, 5.45MB/s]  
21-Apr-2023 15:30:09 DEBUG downloader - Size validation passed
21-Apr-2023 15:30:09 DEBUG downloader - Moving C:\Users\Ovler\AppData\Local\Temp\tmpi1lj_uzn to e:\buct.edu.cn\BUCT-China2023 - 文档\建模专用\ncbi_test\datasets\GSE6269_family.soft.gz
21-Apr-2023 15:30:09 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE6nnn/GSE6269/soft/GSE6269_family.soft.gz
21-Apr-2023 15:30:09 INFO GEOparse - Parsing ./datasets\GSE6269_family.soft.gz: 
21-Apr-2023 15:30:09 DEBUG GEOparse - DATABASE: GeoMiame
21-Apr-2023 15:30:09 DEBUG GEOparse - SERIES: GSE6269
21-Apr-2023 15:30:09 DEBUG GEOparse - PLATFORM: GPL96
21-Apr-2023 15:30:10 DEBUG GEOparse - PLATFORM: GPL570
e:\app\

In [13]:
gse.get_metadata_attribute("title")

'Gene expression patterns in blood leukocytes discriminate patients with acute infections'

In [14]:
# 先把一个个样本的Value值提取出来，然后再转置，这样就可以得到一个样本对应的所有基因的表达值。
sample_value = gse.pivot_samples('VALUE').T
sample_value.tail()

ID_REF,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,1431_at,...,hmm9678-S,hmm9715-S,hmm9738-S,hmm974-S,hmm9750-S,hmm9917-S,hmm9988-S,lysA,pheA,thrB
name,,,,,,,,,,,,,,,,,,,,,
GSM173314,497.6,568.4,224.2,850.1,74.8,1188.4,135.0,13.0,4089.9,18.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173315,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.3,1.0,-5.6,2.8,-6.5,23.0,-5.4,5.4,1.1,-0.6
GSM173316,399.9,499.5,463.8,1119.4,73.8,1429.3,366.1,15.7,18524.4,66.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173317,167.1,827.9,633.7,644.2,63.0,1081.1,46.2,13.4,8046.2,81.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173318,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-3.4,5.0,-1.2,2.9,-12.0,4.2,-7.2,6.9,2.7,-3.3


In [15]:
value_desc = []
Value_Types = [
    'MAS5-calculated Signal intensity',
    'RMA normalized and Log2 transformed'
]
infection = []
n = 0
# value = [] we want 2-d dataframe
import pandas as pd
value = pd.DataFrame()
for i in gse.gsms:
    n += 1


    # 感染状态的处理
    characteristics_ch1 = gse.gsms[i].metadata['characteristics_ch1']   # the infection_status is in the characteristics_ch1
    infection_status = characteristics_ch1[-1].split(':')[1].strip()            # 但characteristics_ch1有的有两个值，有的有三个，取最后一个
    infection.append(infection_status)
    
    # 对数等的处理
    value_type = value_desc.append(gse.gsms[i].columns["description"][1])
    if gse.gsms[i].columns["description"][1] == 'RMA normalized and Log2 transformed':
        # 将sample_value 里第 n 行的每一个值转化为2的x次方
        value = pd.concat([value, 2**sample_value.iloc[n-1]], axis=1)
        # same line as above
    else:
        print(gse.gsms[i].columns["description"][1])
        print('Now just use the raw value')
        print('Samples: ', sample_value.iloc[n-1].head())
        print('-'*50)
        value = pd.concat([value, sample_value.iloc[n-1]], axis=1)

value = value.T

# merge the infection status and value
value.insert(0, 'infection_status', infection)

value.head()

MAS5-calculated Signal intensity
Now just use the raw value
Samples:  ID_REF
1007_s_at    306.2
1053_at      727.1
117_at       998.5
121_at       808.3
1255_g_at      6.3
Name: GSM173175, dtype: float64
--------------------------------------------------
Normalized value from BeadStudio software
Now just use the raw value
Samples:  ID_REF
1007_s_at   NaN
1053_at     NaN
117_at      NaN
121_at      NaN
1255_g_at   NaN
Name: GSM173176, dtype: float64
--------------------------------------------------
MAS5-calculated Signal intensity
Now just use the raw value
Samples:  ID_REF
1007_s_at    313.4
1053_at      540.3
117_at       510.3
121_at       594.6
1255_g_at     12.9
Name: GSM173177, dtype: float64
--------------------------------------------------
MAS5-calculated Signal intensity
Now just use the raw value
Samples:  ID_REF
1007_s_at     371.3
1053_at       183.9
117_at        125.7
121_at       1375.1
1255_g_at       8.2
Name: GSM173178, dtype: float64
--------------------------------

ID_REF,infection_status,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,hmm9678-S,hmm9715-S,hmm9738-S,hmm974-S,hmm9750-S,hmm9917-S,hmm9988-S,lysA,pheA,thrB
GSM173175,Influenza A,306.2,727.1,998.5,808.3,6.3,1385.7,62.4,13.9,6083.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173176,Influenza A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.8,6.4,-0.8,5.1,-6.6,27.3,-7.2,7.2,-0.1,2.1
GSM173177,Influenza B,313.4,540.3,510.3,594.6,12.9,2305.4,142.6,15.4,28163.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173178,"S. aureus, MSSA",371.3,183.9,125.7,1375.1,8.2,754.6,132.6,19.7,5801.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173179,S. pneumoniae,505.6,197.8,310.3,1278.3,53.6,828.9,122.5,103.7,4022.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
# Save the data and Show the first 5 rows
value.to_csv('./datasets/' + GEONAME + '.csv', index=False)
value.head()

ID_REF,infection_status,1007_s_at,1053_at,117_at,121_at,1255_g_at,1294_at,1316_at,1320_at,1405_i_at,...,hmm9678-S,hmm9715-S,hmm9738-S,hmm974-S,hmm9750-S,hmm9917-S,hmm9988-S,lysA,pheA,thrB
GSM173175,Influenza A,306.2,727.1,998.5,808.3,6.3,1385.7,62.4,13.9,6083.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173176,Influenza A,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9.8,6.4,-0.8,5.1,-6.6,27.3,-7.2,7.2,-0.1,2.1
GSM173177,Influenza B,313.4,540.3,510.3,594.6,12.9,2305.4,142.6,15.4,28163.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173178,"S. aureus, MSSA",371.3,183.9,125.7,1375.1,8.2,754.6,132.6,19.7,5801.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GSM173179,S. pneumoniae,505.6,197.8,310.3,1278.3,53.6,828.9,122.5,103.7,4022.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 不用了

不清楚什么时候再把这个id转化成通用的Gene Symbol。

可能在这里就转，也可以分析结束了再转。

大概需要10分钟

In [17]:
# GPL571 = gse.gpls['GPL571']
# n = 0
# for rows in sample_value:
#     Gene_Symbol = GPL571.table[GPL571.table['ID'] == rows]['Gene Symbol']
#     if Gene_Symbol.empty:
#         Gene_Symbol[0] = rows
#     sample_value.rename(columns={rows: Gene_Symbol.values[0]}, inplace=True)
#     n += 1
#     if n % 100 == 0:
#         print("Finished %d rows of %d at %s" % (n, sample_value.shape[1], datetime.datetime.now()))
# sample_value.tail()

In [18]:
# gse.gpls['GPL571'].table.columns # GPL571, 也就是这个GEO数据集的平台，的列。
# # 从这个平台可以直接读取Gene Symbol，Target Description等等有用的东西。

In [19]:
# gse.gpls['GPL571'].table.head() # 前5行